In [2]:
import ISS_decoding.SpaceTx_format as STX
import ISS_decoding.decoding as DEC
import ISS_decoding.qc_metrics as QC

ModuleNotFoundError: No module named 'starfish'

# Specify the path where your sections are saved

In [ ]:
sample = '/media/external/eae_processing/batch_3/R3_B_BOTH_PEAK/'


In [ ]:
# Format the data in SpaceTx format

In [ ]:
STX.make_spacetx_format_zen(path = sample,
codebook_csv = '/home/cml/Downloads/codebook_no_Plp1.csv',
filenames=['Base_1_stitched', 'Base_2_stitched', 
           'Base_3_stitched', 'Base_4_stitched', 
           'Base_5_stitched','Base_6_stitched'],
tile_dim=2000,
pixelscale=0.1625,
channels=["AF750", "Cy5", "Cy3", "AF488", "DAPI"],
DO_decorators=["AF750", "Cy5", "Cy3", "AF488"],
folder_spacetx='SpaceTX_format',
nuclei_channel=5,
)

In [1]:
# Decode every single tile

In [ ]:
DEC.process_experiment(exp_path = sample+'/SpaceTX_format/experiment.json', 
output = sample+'/starfish_output_quality_scores_spots_alignment_new/', 
register = True,
register_dapi = False,
masking_radius = 15, 
threshold = 0.002, 
sigma_vals = [1, 10, 30], # min, max and number
decode_mode = 'PRMC',
normalization_method = 'MH' # or MD
                      )

# Put all the fovs together

# Filter reads

In [ ]:
reads_filt=filter_reads(reads,min_quality_mean=0.2)

# Plot expression

In [ ]:
plot_expression(reads,key='target',colorcode=['red'],xcolumn='xc',ycolumn='yc',genes='individual',size=8,background='black',title_color='white',figuresize=(10,10),save=None,format='pdf')

# Quality metrics

In [ ]:
quality_per_cycle(reads,cycles=6)

In [ ]:
QC.quality_per_gene(reads,on='qcscore_mean',gene_name='target')

In [ ]:
QC.plot_frequencies(reads,on='target')

In [ ]:
compare_scores(reads,score1='quality_mean',score2='quality_minimum',hue='assigned',kind='hist',color='#3266a8')

In [ ]:
plot_scores(reads,on='quality_mean',hue='assigned',log_scale=False) 